In [1]:
import cogent3
from cogent3 import get_app

@cogent3.app.composable.define_app
def renamer(seqs: cogent3.app.typing.UnalignedSeqsType) -> cogent3.app.typing.UnalignedSeqsType:
    """
    A function to rename sequences in a dataset.
    """

    name_map = {
        "homo_sapiens": "Human",
        "pan_troglodytes": "Chimpanzee",
        "gorilla_gorilla": "Gorilla"
    }

    seqs = seqs.rename_seqs(lambda x: name_map.get(x.split("-")[0], x))

    return seqs.take_seqs(list(name_map.values()))


In [2]:
#set folders to read and export data
folder_in = '/xdisk/masel/uliseshmc/EstimatingUd/primates10_114/aligns_cds/'
folder_out = '/xdisk/masel/uliseshmc/EstimatingUd/primates10_114/aligns_cds/codon_aligned'
in_dstore = cogent3.open_data_store(folder_in, suffix='fa', mode='r')
out_dstore = cogent3.open_data_store(folder_out, suffix='fa', mode="w")

#perform a codon alignment and remove stop codons
loader = get_app("load_unaligned", moltype="dna")
rename = renamer()
#why do I trim stop codons? Does this mean that the stop codon is ereased and the rest of the sequence is still analyzed
trim_stops = get_app("trim_stop_codons")
codon_align = get_app("progressive_align", "codon", guide_tree="(Human:0.06,Chimpanzee:0.06,Gorilla:0.1)")
writer = get_app("write_seqs", data_store = out_dstore)
app = loader + rename + trim_stops + codon_align + writer
#underscore is to specify a variable we are not gonna use later
_ = list(app.apply_to(in_dstore[:], parallel = True, show_progress=True))
print(out_dstore.describe)
print(out_dstore.summary_not_completed)

/home/u12/uliseshmc/.conda/envs/EstimatingUd/lib/python3.13/site-packages/cogent3_h5seqs/__init__.py:11: DeprecationWarning: module cogent3.core.new_alignment which will be removed in version 2025.9, use cogent3.core.alignment instead
reason='has been renamed'
  from cogent3.core import new_alignment as c3_alignment
/home/u12/uliseshmc/.conda/envs/EstimatingUd/lib/python3.13/site-packages/cogent3_h5seqs/__init__.py:12: DeprecationWarning: module cogent3.core.new_alphabet which will be removed in version 2025.9, use cogent3.core.alphabet instead
reason='has been renamed'
  from cogent3.core import new_alphabet as c3_alphabet
/home/u12/uliseshmc/.conda/envs/EstimatingUd/lib/python3.13/site-packages/cogent3_h5seqs/__init__.py:13: DeprecationWarning: module cogent3.core.new_moltype which will be removed in version 2025.9, use cogent3.core.moltype instead
reason='has been renamed'
  from cogent3.core import new_moltype as c3_moltype
/home/u12/uliseshmc/.conda/envs/EstimatingUd/lib/python3.1

   0%|          |00:00<?

Directory datastore
record type      number
-----------------------
completed          1694
not_completed       444
logs                 12
-----------------------
not completed records
type     origin     message                                          num    source                                     
-----------------------------------------------------------------------------------------------------------------------
ERROR    renamer    'ValueError: non-uniq...> at 0x7f60a76300e0>'    444    ENSG00000243480.fa, ENSG00000185668.fa, ...
-----------------------------------------------------------------------------------------------------------------------
